```mermaid
graph LR

    path -->|"openfile()"|rowlist
```

In [1]:
import os
from tqdm.notebook import tqdm, trange
import time
import re
os.getcwd()

'c:\\Workspace\\Tutorial\\BGCA\\thai_data'

In [2]:
SENTIMENT_TOKEN = ['<neu>','<pos>','<neg>']
OPINION_TOKEN = ['<opinion>']

SPECIAL_TOKENS = SENTIMENT_TOKEN + OPINION_TOKEN
# TOKENS

In [3]:
from googletrans import Translator
translator = Translator()

def openfile(path: str)->list:
    with open(path, 'r') as file:
        lines = file.readlines()
    return [line.replace('\n','') for line in lines]


def input_output_pair(text:str):
    input, output = text.split(' ===> ')
    return (input, output)

def translate(text:str, sorce_lang='en', dest_lang='th'):
    translations = translator.translate(text, dest=dest_lang)
    return translations.text

In [31]:
txt_file_path = 'laptop14/en/train_aste_processed.txt'
row_list = openfile(txt_file_path)


In [4]:
def split_label(text:str, keep_tag=True):
    if keep_tag:
        pattern = r'(<neu>|<pos>|<neg>|<opinion>)'
    else:
        pattern = r'<neu>|<pos>|<neg>|<opinion>'

    split_text = re.split(pattern, text)
    filtered_split_text = [s.strip() for s in split_text if s.strip()]

    return filtered_split_text

# method 1: translater entire row `( text ==> label)`

In [28]:


translation_list = []
for row in tqdm(row_list, desc='Translating'):
    translation_list.append(translate(row))


Translating:   0%|          | 0/906 [00:00<?, ?it/s]

# method 2: translater split by input, output by **googletrans**

In [5]:
def dummy_translate(text:str, sorce_lang='en', dest_lang='th'):
    return f'to_{dest_lang}({text})'

In [6]:
txt_file_path = 'laptop14/en/train_aste_processed.txt'
row_list = openfile(txt_file_path)

In [7]:
textfiles = [
                # 'rest14/en/train_aste_processed.txt',
                # 'rest14/en/dev_aste_processed.txt',
                # 'rest14/en/test_aste_processed.txt',
                # 'laptop14/en/train_aste_processed.txt',
                # 'laptop14/en/dev_aste_processed.txt',
                'laptop14/en/test_aste_processed.txt',
]

In [8]:
for input_file in textfiles:
    folder_path, filename = os.path.split(input_file)
    print(f"Folder path: {folder_path}")
    print(f"Filename: {filename}")

    save_file_path = f'{folder_path[:-2]}th/{filename}'
    print(f"SAVE Destiantion: {save_file_path}")
    print('----------------------------------------')

Folder path: laptop14/en
Filename: test_aste_processed.txt
SAVE Destiantion: laptop14/th/test_aste_processed.txt
----------------------------------------


In [9]:
for input_file in tqdm(textfiles, desc=f'{input_file}'):
    folder_path, filename = os.path.split(input_file)
    print(f"Folder path: {folder_path}")
    print(f"Filename: {filename}")
    row_list = openfile(input_file)

    translation_list = []
    for row in tqdm(row_list, desc=f'Translating (METHOD 2)'):
        # print(row)
        input, output = input_output_pair(row)
        output = split_label(output, keep_tag=True)
        # print(input, '-->', output)

        # trans_input = dummy_translate(input)
        trans_input = translate(input)

        trans_output = ''
        for token in output:
            if token not in SPECIAL_TOKENS:
                # trans_output += dummy_translate(token)
                trans_output += translate(token)
            else:
                trans_output += token
            trans_output += ' '

        # print(trans_input, '~~>', trans_output)
        translation_list.append(trans_input + ' ===> '+ trans_output)
        # print('------------------------------------------------')

    # translation_list
    save_file_path = f'{folder_path[:-2]}th/{filename}'
    if not os.path.exists(save_file_path):

        with open(save_file_path, 'w') as file:
            for line in translation_list:
                file.write(line + '\n')
    else:
        print(f"File '{save_file_path}' already exists. Not overwriting.")

laptop14/en/test_aste_processed.txt:   0%|          | 0/1 [00:00<?, ?it/s]

Folder path: laptop14/en
Filename: test_aste_processed.txt


Translating (METHOD 2):   0%|          | 0/328 [00:00<?, ?it/s]

In [65]:
translation_list = []
for row in tqdm(row_list, desc=f'Translating (METHOD 2) {txt_file_path}'):
    # print(row)
    input, output = input_output_pair(row)
    output = split_label(output, keep_tag=True)
    # print(input, '-->', output)

    # trans_input = dummy_translate(input)
    trans_input = translate(input)

    trans_output = ''
    for token in output:
        if token not in SPECIAL_TOKENS:
            # trans_output += dummy_translate(token)
            trans_output += translate(token)
        else:
            trans_output += token
        trans_output += ' '

    # print(trans_input, '~~>', trans_output)
    translation_list.append(trans_input + ' ===> '+ trans_output)
    # print('------------------------------------------------')

translation_list

Translating (METHOD 2) laptop14/en/train_aste_processed.txt:   0%|          | 0/906 [00:00<?, ?it/s]

['ฉันชาร์จมันในเวลากลางคืนและข้ามสายไฟติดตัวไปด้วยเพราะอายุการใช้งานแบตเตอรี่ที่ดี ===> <pos> อายุการใช้งานแบตเตอรี่ <opinion> ดี ',
 'มันมีคุณภาพสูงมี GUI นักฆ่ามีความเสถียรมากสามารถขยายได้สูงมีการใช้งานที่ดีมากใช้งานง่ายใช้งานง่ายและงดงามอย่างแน่นอน ===> <pos> คุณภาพ <opinion> สูง <pos> GUI <opinion> นักฆ่า <pos> แอปพลิเคชัน <opinion> ดี <pos> ใช้ <opinion> ง่าย ',
 'ง่ายต่อการเริ่มต้นและไม่ร้อนเกินไปเท่าแล็ปท็อปอื่น ๆ ===> <pos> เริ่มต้น <opinion> ง่าย ',
 'แล็ปท็อปที่ยอดเยี่ยมที่มีคุณสมบัติที่ยอดเยี่ยมมากมาย! ===> <pos> คุณสมบัติ <opinion> ยอดเยี่ยม ',
 'คืนหนึ่งฉันปิดสิ่งที่น่าประหลาดใจหลังจากใช้มันในวันถัดไปฉันเปิดเครื่องไม่มี GUI หน้าจอมืดทั้งหมดไฟคงที่ไฟฮาร์ดไดรฟ์คงที่และไม่กระพริบเหมือนปกติ ===> <neg> GUI <opinion> เลขที่ <neg> หน้าจอ <opinion> มืด <neu> แสงไฟ <opinion> สม่ำเสมอ <neg> ไฟฮาร์ดไดรฟ์ <opinion> สม่ำเสมอ ',
 'อย่างไรก็ตามท่าทางแบบมัลติทัชและพื้นที่ติดตามขนาดใหญ่ทำให้ไม่จำเป็นต้องใช้เมาส์ภายนอก (เว้นแต่คุณจะเล่นเกมอีกครั้ง) ===> <neu> เมาส์ภายนอก <opinion> ไม่จำเป็น

# method 3: translater split by input, output by **pythainlp.translate**

In [1]:
textfiles = [
                'laptop14/en/train_aste_processed.txt',
                'laptop14/en/dev_aste_processed.txt',
                'laptop14/en/test_aste_processed.txt',
                'rest14/en/train_aste_processed.txt',
                'rest14/en/dev_aste_processed.txt',
                'rest14/en/test_aste_processed.txt',
]

In [2]:
from pythainlp.translate import Translate
th2en = Translate('th', 'en')

th2en.translate("ฉันรักแมว")
# output: I love cat.

ModuleNotFoundError: No module named 'fairseq'

In [1]:
from pythainlp.translate import EnThTranslator

enth = EnThTranslator()

enth.translate("I love cat.")

ImportError: cannot import name 'EnThTranslator' from 'pythainlp.translate' (c:\Users\JINNII-CustomPC\miniconda310\lib\site-packages\pythainlp\translate\__init__.py)

# SAVE FILE

In [66]:
savefile_path = 'laptop14/th/train_aste_processed_v2.txt'
if not os.path.exists(savefile_path):

    with open(savefile_path, 'w') as file:
        for line in translation_list:
            file.write(line + '\n')
else:
    print(f"File '{savefile_path}' already exists. Not overwriting.")